# NASAstronauts
An astronaut is a human being who travels in space or is transported by something that does it. Thanks to this dataset we can know many curiosities about these "magic" people. This information relates to the generalities of astronauts from 1959 to 2017. In fact, in April 9, 1959, nine astronauts were selected for the Mercury Progra, one of the firsts SPACE programs.

## Dataset
This dataset (astronauts.csv) has nineteen columns, organized as follows: Name, Year, Group, Status, Birthdate, Birthplace, Gender, Alma mater, Undergraduate and Graduate Major.

## Question 
1) What is the average age of astronauts on duty? <br />
2) What are the most common academic studies? <br />
3) Where the astronauts mostly born? <br />
4) What are the astronauts with more space walks? With how many hours? <br />
5) What are the astronauts with more space flights? <br />
6) What is the average age of death? <br />

## Let's start
First of all, we need to import all the modules, in particular Pandas and Numpy for the data manipulation, then Matplotlib and Seaborn for the data visualization.

In [28]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from datetime import date
from dateutil.relativedelta import relativedelta

Then we need to import the dataset that we'll use

In [6]:
df = pd.read_csv('CSV/astronauts.csv')

Let's see the first input

In [59]:
df.head(1)

,Name,Year,Group,Status,Birth Date,Birth Place,Gender,Alma Mater,Undergraduate Major,Graduate Major,Military Rank,Military Branch,Space Flights,Space Flight (hr),Space Walks,Space Walks (hr),Missions,Death Date,Death Mission
0,Joseph M. Acaba,2004.0,19.0,Active,5/17/1967,"Inglewood, CA",Male,University of California-Santa Barbara; Univer...,Geology,Geology,NaN,NaN,2,3307,2,13.0,"STS-119 (Discovery), ISS-31/32 (Soyuz)",NaN,NaN


### 1) What is the average age of astronauts on duty? 
We need to focus just on the records with Status = "Active", and then we can calculate the average. We suppose for simplicity that they're currently active in 2020. We'll use another dataframe with just the born date of active astronauts.

In [48]:
df_years = df[df["Status"] == "Active"]
df_years = df_years[['Birth Date']]
df_years.head(3)

,Birth Date
0,5/17/1967
11,8/23/1967
15,11/26/1963


Before going on, we have to check in there is any null value

In [60]:
df_years.isnull().sum()

Birth Date    0
Age           0
dtype: int64

Perfect, there is no null value, now we can add another column with the equivalent ages. 

In [56]:
end_date = datetime.now()
df_years["Age"] = df_years.apply(lambda x: 
                                     relativedelta(end_date, 
                                                   datetime.strptime(x["Birth Date"], '%m/%d/%Y')
                                ).years, axis=1)

df_years.head(3)

,Birth Date,Age
0,5/17/1967,53
11,8/23/1967,52
15,11/26/1963,56


In [61]:
df_years["Age"].mean()

54.34

**So the average age of astronauts on duty is 53.34 years!**

### 2) What are the most common academic studies? 